In [ ]:
from datashader.transfer_functions import shade, stack
from datashader.colors import Elevation
from pystac_client import Client
from xrspatial import hillshade

import planetary_computer as pc
import xarray as xr
import pandas as pd

In [7]:
def get_elevation(areas_of_interest):
    catalog = Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
    nasadem = catalog.search(collections=["nasadem"], intersects=areas_of_interest)

    items = [item for item in nasadem.get_items()]
    print(f"Returned {len(items)} Items")

    item = items[0]
    signed_asset = pc.sign(item.assets["elevation"])
    da = xr.open_rasterio(signed_asset.href)
    return float(da.mean()), float(da.std()) ** 2

In [ ]:
from tqdm import tqdm
grid_data = pd.read_csv('../data/grid_metadata.csv')
wkt = grid_data['wkt'].values
elevation_mean = []
elevation_var = []


for i in tqdm(range(len(wkt))):
    geometry = wkt[i]
    geometry = geometry.replace('(', '', -1)
    geometry = geometry.replace(')', '', -1)
    geometry = geometry.replace(',', '', -1)
    geometry = list(map(float, geometry.split()[1:]))
    geometry = [geometry[i:i+2] for i in range(0, len(geometry), 2)]

    areas_of_interest = {"type": "Polygon", "coordinates": [geometry]}
    m, v = get_elevation(areas_of_interest)

    elevation_mean.append(m)
    elevation_var.append(v)

In [ ]:
grid_data['elevation_mean'] = elevation_mean
grid_data['elevation_var'] = elevation_var

In [ ]:
grid_data.to_csv('../data/proc/grid_nasadem.csv', index=False)